**PyTorch Model AWS Deployment Example**

Train a PyTorch model

In [52]:
import torch

import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:
X_test

array([[ 0.35451684, -0.58505976,  0.55777524,  0.02224751],
       [-0.13307079,  1.65083742, -1.16139502, -1.17911778],
       [ 2.30486738, -1.0322392 ,  1.8185001 ,  1.49058286],
       [ 0.23261993, -0.36147005,  0.44316389,  0.4227026 ],
       [ 1.2077952 , -0.58505976,  0.61508092,  0.28921757],
       [-0.49876152,  0.75647855, -1.27600637, -1.04563275],
       [-0.2549677 , -0.36147005, -0.07258719,  0.15573254],
       [ 1.32969211,  0.08570939,  0.78699794,  1.49058286],
       [ 0.47641375, -1.92659808,  0.44316389,  0.4227026 ],
       [-0.01117388, -0.80864948,  0.09932984,  0.02224751],
       [ 0.84210448,  0.30929911,  0.78699794,  1.09012776],
       [-1.23014297, -0.13788033, -1.33331205, -1.44608785],
       [-0.37686461,  0.98006827, -1.39061772, -1.31260282],
       [-1.10824606,  0.08570939, -1.27600637, -1.44608785],
       [-0.86445224,  1.65083742, -1.27600637, -1.17911778],
       [ 0.59831066,  0.53288883,  0.55777524,  0.55618763],
       [ 0.84210448, -0.

In [80]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [81]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [25]:
import torch.nn as nn

class NNModel(nn.Module):
    def __init__(self):
        input_dim = 4
        output_dim = 3
        super(NNModel, self).__init__()
        self.input_layer    = nn.Linear(input_dim, 128)
        self.hidden_layer1  = nn.Linear(128, 64)
        self.output_layer   = nn.Linear(64, output_dim)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()
    
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

In [26]:
model = NNModel()

learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

Train for 10 epochs

In [35]:
num_epochs = 10

for epoch in range(num_epochs):
    #clear out the gradients from the last step loss.backward()
    optimizer.zero_grad()
    
    #forward feed
    output_train = model(X_train)

    #calculate the loss
    loss_train = criterion(output_train, y_train)
    
    #backward propagation: calculate gradients
    loss_train.backward()

    #update the weights
    optimizer.step()

    output_test = model(X_test)
    loss_test = criterion(output_test,y_test)

    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}, Test Loss: {loss_test.item():.4f}")

Epoch 2/10, Train Loss: 0.0463, Test Loss: 0.0120
Epoch 4/10, Train Loss: 0.0450, Test Loss: 0.0093
Epoch 6/10, Train Loss: 0.0432, Test Loss: 0.0094
Epoch 8/10, Train Loss: 0.0405, Test Loss: 0.0112
Epoch 10/10, Train Loss: 0.0389, Test Loss: 0.0127


In [33]:
predictions_train = []
predictions_test =  []
with torch.no_grad():
    predictions_train = model(X_train)
    predictions_test = model(X_test)

tensor([[ -3.7505,   2.8322,   1.2186],
        [  8.1387,  -2.7037,  -7.5045],
        [-11.3830,   3.8642,   9.5603],
        [ -3.6804,   2.2939,   1.6552],
        [ -5.6257,   3.4858,   2.8000],
        [  6.7800,  -1.9779,  -6.6254],
        [ -1.4664,   2.1188,  -0.7140],
        [ -7.1113,   2.3506,   6.0630],
        [ -6.7089,   4.0703,   3.1185],
        [ -2.8905,   2.8437,   0.1573],
        [ -5.3851,   2.0424,   4.3183],
        [  6.8727,  -1.6656,  -7.2659],
        [  7.7975,  -2.4223,  -7.4617],
        [  7.0649,  -1.9088,  -7.2152],
        [  9.1169,  -3.3577,  -8.0306],
        [ -3.5266,   2.0687,   2.1146],
        [ -6.7714,   1.9784,   5.9019],
        [ -3.1669,   3.2909,  -0.0600],
        [ -3.0412,   2.4505,   0.7630],
        [ -6.8142,   2.0830,   5.6052],
        [  7.4494,  -2.2848,  -7.2649],
        [ -4.4545,   1.9013,   3.0549],
        [  7.2757,  -2.3487,  -6.8825],
        [ -6.6267,   2.1347,   5.3197],
        [ -8.1050,   3.1814,   7.1337],


Model evaluation on training and test set

In [30]:
import numpy as np

def get_accuracy_multiclass(pred_arr,original_arr):
    if len(pred_arr)!=len(original_arr):
        return False
    pred_arr = pred_arr.numpy()
    original_arr = original_arr.numpy()
    final_pred= []
    # we will get something like this in the pred_arr [32.1680,12.9350,-58.4877]
    # so will be taking the index of that argument which has the highest value here 32.1680 which corresponds to 0th index
    for i in range(len(pred_arr)):
        final_pred.append(np.argmax(pred_arr[i]))
    final_pred = np.array(final_pred)
    count = 0
    #here we are doing a simple comparison between the predicted_arr and the original_arr to get the final accuracy
    for i in range(len(original_arr)):
        if final_pred[i] == original_arr[i]:
            count+=1
    return count/len(final_pred)

In [31]:
train_acc = get_accuracy_multiclass(predictions_train,y_train)
test_acc  = get_accuracy_multiclass(predictions_test,y_test)

print(f"Training Accuracy: {round(train_acc*100,3)}")
print(f"Test Accuracy: {round(test_acc*100,3)}")

Training Accuracy: 95.833
Test Accuracy: 96.667


Save the model to the local path

In [36]:
PATH = 'pytorch_model/model.pt'
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(PATH)

Zip and upload the model to S3

In [74]:
!tar -czf pytorch_model.tar.gz pytorch_model
!aws s3 cp pytorch_model.tar.gz s3://ml-deployment-thomas/pytorch_model.tar.gz


upload: ./pytorch_model.tar.gz to s3://ml-deployment-thomas/pytorch_model.tar.gz


Deploy the model to SageMaker

In [78]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data='s3://ml-deployment-thomas/pytorch_model.tar.gz', 
                             role='SageMakerRole',
                             entry_point='./pytorch_model/code/inference.py',
                             framework_version='2.1',
                             py_version='py310')

predictor = pytorch_model.deploy(instance_type='ml.m5.xlarge', initial_instance_count=1)

-----!

Cleaning: remove the endpoint and model on AWS

In [77]:
predictor.delete_endpoint()
pytorch_model.delete_model()